In [1]:
import pandas as pd
import requests


In [2]:
df= pd.read_csv("combined_output.csv")

In [3]:
df

,term,Response Code,Returned List,count
0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0
1,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0
2,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0
3,0,200.0,[],12.0
4,00,NaN,NaN,1.0
...,...,...,...,...
51185,zot,200.0,[],1.0
51186,zs95f,200.0,[],1.0
51187,zur,200.0,[],1.0
51188,zw315m,200.0,[],1.0


In [4]:
df_returned_list= df[df['Returned List'].apply(lambda x: len(str(x))>20)]


In [5]:
df_returned_list

,term,Response Code,Returned List,count
0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0
1,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0
2,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0
5328,1,200.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0
5329,1,200.0,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0
...,...,...,...,...
49998,997,200.0,[{'broader': [{'uri': 'http://dewey.info/class...,9.0
49999,997.11,200.0,"[{'created': '2000-02-02', 'modified': '2018-0...",1.0
50008,998,200.0,[{'prefLabel': {'de': 'Inseln des Arktischen O...,15.0
50009,998.2,200.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0


In [6]:

# Function to split the term and create new rows
def split_term(row):
    terms = list(str(row['term']))
    new_terms = [''.join(terms[:i+1]) for i in range(len(terms))]
    new_rows = []
    for term in new_terms[:-1]:
        new_row = row.copy()
        if term[-1]==".":
            continue
        new_row['term'] = term
        for col in new_row.index:
            if col != 'term':
                new_row[col] = None
        new_row["origional term"]= row['term']
        new_rows.append(new_row)
    row["origional term"]= row['term'] 
    new_rows.append(row)
    return new_rows

# Apply the function to the DataFrame
new_rows = []
for _, row in df_returned_list.iterrows():
    new_rows.extend(split_term(row))

# Create a new DataFrame from the new rows
new_df = pd.DataFrame(new_rows)



In [7]:
new_df.set_index('origional term', inplace=True)
new_df.sort_index(inplace=True)
new_df

,term,Response Code,Returned List,count
origional term,,,,
0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0
0,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0
0,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0
1,1,200.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0
1,1,200.0,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0
...,...,...,...,...
998.2,998,NaN,None,NaN
998.2,998.2,200.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0
999,9,NaN,None,NaN


In [8]:
def get_concept_from_term(term):
    voc = "http://dewey.info/scheme/edition/e23/"
    url = f"https://coli-conc.gbv.de/api/concepts?notation={term}&voc={voc}"
    
    response = requests.get(url)
    if response.status_code == 200:
            data = response.json()
            return data
    else: 
         return response.status_code

In [9]:
# new_df['ReturnedListForGAP'] = new_df.term.map(get_concept_from_term)

In [10]:
new_df.to_csv("formated and requested to find gap.csv")

In [11]:
new_df

,term,Response Code,Returned List,count,ReturnedListForGAP
origional term,,,,,
0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo..."
0,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo..."
0,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo..."
1,1,200.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo..."
1,1,200.0,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo..."
...,...,...,...,...,...
998.2,998,NaN,None,NaN,[{'prefLabel': {'de': 'Inseln des Arktischen O...
998.2,998.2,200.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':..."
999,9,NaN,None,NaN,"[{'notation': ['9'], 'topConceptOf': [{'uri': ..."


In [21]:
new_df.reset_index(inplace=True)

In [19]:
# new_df.set_index(["origional term","term"], inplace=True)

In [29]:
grouped = new_df.groupby('origional term')['term'].apply(lambda x: (x.astype(str).str.len() < 20).any())


In [24]:
new_df['HasGap'] = new_df['origional term'].map(grouped)


In [57]:
new_df

,origional term,origional term.1,term,Response Code,Returned List,count,ReturnedListForGAP,HasGap
0,0.0,0.0,0.0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
1,0.0,0.0,0.0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
2,0.0,0.0,0.0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
3,1.0,1.0,1.0,200.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",False
4,1.0,1.0,1.0,200.0,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",False
...,...,...,...,...,...,...,...,...
56582,998.2,998.2,998.0,NaN,NaN,NaN,[{'prefLabel': {'de': 'Inseln des Arktischen O...,False
56583,998.2,998.2,998.2,200.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",False
56584,999.0,999.0,9.0,NaN,NaN,NaN,"[{'notation': ['9'], 'topConceptOf': [{'uri': ...",False
56585,999.0,999.0,99.0,NaN,NaN,NaN,"[{'modified': '2018-09-19', 'created': '2000-0...",False


In [31]:
new_df.set_index("origional term", inplace=True)

In [41]:
new_df

,term,Response Code,Returned List,count,ReturnedListForGAP
origional term,,,,,
0,0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo..."
0,0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo..."
0,0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo..."
1,1,200.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo..."
1,1,200.0,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo..."
...,...,...,...,...,...
998.2,998,NaN,None,NaN,[{'prefLabel': {'de': 'Inseln des Arktischen O...
998.2,998.2,200.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':..."
999,9,NaN,None,NaN,"[{'notation': ['9'], 'topConceptOf': [{'uri': ..."


In [52]:
def check_for_gap(group):
    if (group["ReturnedListForGAP"].apply(lambda x: len(str(x))<20).any()):
        group["HasGap"]= True
    else:
        group["HasGap"]= False

    return group

new_df= new_df.groupby(new_df.index).apply(check_for_gap)
    

In [55]:
new_df

,origional term,origional term.1,term,Response Code,Returned List,count,ReturnedListForGAP,HasGap
0,0.0,0.0,0.0,200.0,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
1,0.0,0.0,0.0,200.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
2,0.0,0.0,0.0,200.0,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
3,1.0,1.0,1.0,200.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",False
4,1.0,1.0,1.0,200.0,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",False
...,...,...,...,...,...,...,...,...
56582,998.2,998.2,998.0,NaN,NaN,NaN,[{'prefLabel': {'de': 'Inseln des Arktischen O...,False
56583,998.2,998.2,998.2,200.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",False
56584,999.0,999.0,9.0,NaN,NaN,NaN,"[{'notation': ['9'], 'topConceptOf': [{'uri': ...",False
56585,999.0,999.0,99.0,NaN,NaN,NaN,"[{'modified': '2018-09-19', 'created': '2000-0...",False


In [61]:
new_df["Response Code"] = new_df["Response Code"].fillna(404)
new_df["Response Code"] = new_df["Response Code"].astype(int)

,origional term,origional term.1,term,Response Code,Returned List,count,ReturnedListForGAP,HasGap
0,0.0,0.0,0.0,200,"[{'notation': ['000'], 'inScheme': [{'uri': 'h...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
1,0.0,0.0,0.0,200,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
2,0.0,0.0,0.0,200,"[{'notation': ['00'], 'inScheme': [{'uri': 'ht...",12.0,"[{'uri': 'http://dewey.info/class/0/e23/', 'mo...",False
3,1.0,1.0,1.0,200,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",False
4,1.0,1.0,1.0,200,"[{'notation': ['01'], 'inScheme': [{'uri': 'ht...",6342.0,"[{'uri': 'http://dewey.info/class/1/e23/', 'mo...",False
...,...,...,...,...,...,...,...,...
56582,998.2,998.2,998.0,404,NaN,NaN,[{'prefLabel': {'de': 'Inseln des Arktischen O...,False
56583,998.2,998.2,998.2,200,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",1.0,"[{'prefLabel': {'de': 'Grönland'}, 'inScheme':...",False
56584,999.0,999.0,9.0,404,NaN,NaN,"[{'notation': ['9'], 'topConceptOf': [{'uri': ...",False
56585,999.0,999.0,99.0,404,NaN,NaN,"[{'modified': '2018-09-19', 'created': '2000-0...",False
